# Base de données: interroger une base de données MySQL

## Imports

In [7]:
import os # exécuter du bash dans l'os?
import sqlite3 # exécuter du sql avec python grâce à sqlite3
import zipfile # pour dézipper
from pathlib import Path # se promener dans l'arborescence des fichiers
import requests # pour adresser sur le web

# Créer un dossier data avec 'mkdir' si le chemin n'existe pas encore

if not os.path.exists("../data"):
    os.mkdir("../data")

## Créer une base de données SQLite

In [10]:
# Créer un répertoire DB s'il n'existe pas encore

db_path = '../data/db'
nom_variable = '../data/essai'

if not os.path.exists(nom_variable):
    os.mkdir(nom_variable)

In [14]:
# Récupérer le fichier ZIP qui contient la DB de test

url = "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip"

filename = url.split("/")[-1] # préparer le nom du fichier à partir de l'url en parcourant à l'envers et en prenant le dernier élément
filename2 = url.split("/")[-2]

print(filename)
print(filename2)

# Récupérer le fichier zip dans la RAM

response = requests.get(url) # adressage avec 'get'

chinook.zip
03


In [4]:
# Ecrire le fichier sur le disque

with open(os.path.join(db_path, filename), 'wb') as f: # avec la fonction 'open' qui joint le nom du fichier à l'endroit d'écriture, définie comme 'f', écrire le contenu de 'response' qui chope l'url
    f.write(response.content)

In [15]:
help(open) # 'w' pour écrire le contenu, 'b' en mode binaire

Help on built-in function open in module io:

open(file, mode='r', buffering=-1, encoding=None, errors=None, newline=None, closefd=True, opener=None)
    Open file and return a stream.  Raise OSError upon failure.
    
    file is either a text or byte string giving the name (and the path
    if the file isn't in the current working directory) of the file to
    be opened or an integer file descriptor of the file to be
    wrapped. (If a file descriptor is given, it is closed when the
    returned I/O object is closed, unless closefd is set to False.)
    
    mode is an optional string that specifies the mode in which the file
    is opened. It defaults to 'r' which means open for reading in text
    mode.  Other common values are 'w' for writing (truncating the file if
    it already exists), 'x' for creating and writing to a new file, and
    'a' for appending (which on some Unix systems, means that all writes
    append to the end of the file regardless of the current seek position

In [5]:
# Extraire le zip pour obtenir la db

with zipfile.ZipFile(os.path.join(db_path, filename), 'r') as zip_ref: # lire le contenu du fichier zip et extraire le contenu dans le dossier 'db' et lui donner le nom du fichier voulu
    zip_ref.extractall(db_path)

## Schéma de la base de données

<img src="https://cdn.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg" />

## Se connecter à la base de données

In [6]:
conn = sqlite3.connect(os.path.join(db_path, "chinook.db")) # stocker la requête de connexion dans une variable
c = conn.cursor() # nommer le curseur 

## Trouver l'identifiant de Miles Davis

In [7]:
artist = 'Miles Davis'

query1 = f"""
    SELECT
        artistId
    FROM
        artists
    WHERE
        name = '{artist}';
"""
c.execute(query1)
artist_id = c.fetchone()[0]

print(artist_id)

68


## Trouver les identifiants d'albums de Miles Davis

In [8]:
query2 = f"""
    SELECT
        albumId
    FROM 
        albums 
    WHERE 
        artistId = '{artist_id}';
"""
albums_ids = []
for row in c.execute(query2):
    albums_ids.append(str(row[0]))

print(albums_ids)

['48', '49', '157']


## Trouver les morceaux des albums de Miles Davis

In [9]:
query3 = f"""
    SELECT
        name,
        albumId
    FROM
        tracks
    WHERE 
        albumId IN ({",".join(albums_ids)});
"""

songs = set()
for row in c.execute(query3):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)


36 distinct songs found:

'Round Midnight
Black Satin
Blues For Pablo
Blues For Pablo (Alternate Take)
Bye Bye Blackbird
Compulsion
E.S.P.
Generique
I Don't Wanna Be Kissed (By Anyone But You)
I Don't Wanna Be Kissed (By Anyone But You) (Alternate Take)
Jean Pierre (Live)
Jeru
Lament
Little Church (Live)
Miles Ahead
Miles Runs The Voodoo Down
My Funny Valentine (Live)
My Ship
Nefertiti
New Rhumba
Now's The Time
Petits Machins (Little Stuff)
Portia
So What
Someday My Prince Will Come
Springsville
Springsville (Alternate Take)
Summertime
Tempus Fugit
The Duke
The Maids Of Cadiz
The Meaning Of The Blues
The Meaning Of The Blues/Lament (Alternate Take)
The Pan Piper
Time After Time
Walkin'


## En une seule requête...

In [10]:
query4 = f"""
    SELECT
        tracks.name,
        tracks.albumId
    FROM
        artists
        JOIN albums ON artists.artistId = albums.artistId
        JOIN tracks ON albums.albumId = tracks.albumId
    WHERE 
        artists.name = '{artist}';
"""

songs = set()
for row in c.execute(query4):
    songs.add(row[0])

print(f"\n{len(songs)} distinct songs found:\n")
for song in sorted(songs):
    print(song)


36 distinct songs found:

'Round Midnight
Black Satin
Blues For Pablo
Blues For Pablo (Alternate Take)
Bye Bye Blackbird
Compulsion
E.S.P.
Generique
I Don't Wanna Be Kissed (By Anyone But You)
I Don't Wanna Be Kissed (By Anyone But You) (Alternate Take)
Jean Pierre (Live)
Jeru
Lament
Little Church (Live)
Miles Ahead
Miles Runs The Voodoo Down
My Funny Valentine (Live)
My Ship
Nefertiti
New Rhumba
Now's The Time
Petits Machins (Little Stuff)
Portia
So What
Someday My Prince Will Come
Springsville
Springsville (Alternate Take)
Summertime
Tempus Fugit
The Duke
The Maids Of Cadiz
The Meaning Of The Blues
The Meaning Of The Blues/Lament (Alternate Take)
The Pan Piper
Time After Time
Walkin'


## Pour en savoir plus

- Tutoriel SQLite : https://www.sqlitetutorial.net/